# Analyse GGC data

Analyse assigned Brinkman keywords (Brinkeys) and create vocab dataset for training Annif.

In [3]:
import os
import csv
import pandas as pd
import numpy as np

In [4]:
# Import GGC-data
df = pd.read_csv('data/vraag_20190620.txt', sep='\t')
print('Number of rows: ' + str(len(df)))
df.head()

Number of rows: 12243


,maa1,maa2,ppn,jvu_1100,taal_1500_publ,taal_1500_orig,isbn_2000,unesco_1121,unesco_1122,nur_codes_5061,brinkman_520x,brinkman_520x_ppn,onix_7880,prim_auteur_3000,sec_auteur_3011,titel_4000,ondertitel_4000,samenvatting_4207
0,A,Aa,322079640,2015,ned,NaN,9.789054e+12,7,NaN,321,levensbeschrijvingen,075613816,9789054292692,Han/van@Bree$aut$!069567727!Han van Bree 1957-,NaN,De @geest van het Oude Loo,Juliana en haar vriendenkring 1947-1957,In het boek wordt de hofcrisis van 1956 voor h...
1,A,Aa,33015673X,2015,ned,fra,9.789490e+12,b,NaN,736,perceptie | esthetiek,075618451 | 075605503,9789490334086,Gilles@Deleuze$aut$!06860873X!Gilles Deleuze 1...,Walter/van der@Star$trl$!125379315!Walter van ...,@Francis Bacon,logica van de gewaarwording,Esthetische analyse van het werk van de Britse...
2,A,Aa,352655844,2015,ned,NaN,9.789460e+12,7,NaN,698,levensbeschrijvingen,075613816,9789460041228,Anton/van de@Sande$aut$!067525180!Antonius Wil...,NaN,@Prins Frederik der Nederlanden 1797-1881,gentleman naast de troon,"Beschrijving van het leven van prins Frederik,..."
3,A,Aa,352699566,2015,ned,NaN,9.789462e+12,z,NaN,648,vakantieverblijven,075625156,9789462080744,Mieke@Dings$aut$!270022139!Mieke Dings 1979-,NaN,@Tussen tent en villa,het vakantiepark in Nederland 1920-nu,Het vakantiepark: wie heeft er niet weleens ee...
4,A,Aa,362837317,2015,ned,NaN,9.789047e+12,4,NaN,301,romans en novellen ; oorspr. - Nederlands,075629402,9789046815809,Jan/van der@Mast$aut$!07502943X!Jan van der Ma...,NaN,@Agneta,NaN,"Jacques van Marken (1845-1906), oprichter van ..."


## Exploratory research
Before we create a dataset we have to analyse our data.

In [12]:
# Brinkeys value_counts - check which Brinkey or combination of Brinkeys is assigned the most.
brinkman_termen = df['brinkman_520x'].value_counts().reset_index()

print('Number of times each Brinkey or combination of Brinkeys is assigned:\n')
print(brinkman_termen.head(20))

Number of times each Brinkey or combination of Brinkeys is assigned:

                                                index  brinkman_520x
0                       romans en novellen ; vertaald           2165
1           romans en novellen ; oorspr. - Nederlands           1960
2                              jeugdboeken ; verhalen           1265
3                                levensbeschrijvingen            193
4                    gedichten ; oorspr. - Nederlands            181
5                                    autobiografieën             99
6                                             columns             61
7                                         levenskunst             55
8                                       stripverhalen             48
9              jeugdboeken ; verhalen | prentenboeken             46
10                                       geloofsleven             41
11  jeugdboeken ; verhalen | romans en novellen ; ...             39
12                jeugdboeken ; in

In [4]:
# See value_counts for each individual Brinkey.
brinkman_split = df['brinkman_520x'].str.split('|').apply(pd.Series).reset_index().melt(id_vars='index').dropna()[['index', 'value']].set_index('index')
brinkman_split = brinkman_split['value'].str.strip().reset_index()

print('Number of times each individual Brinkey is assigned:\n')
print(brinkman_split.iloc[:, 1].value_counts().head(15))

Number of times each Brinkey is assigned:

romans en novellen ; vertaald                2238
romans en novellen ; oorspr. - Nederlands    2033
jeugdboeken ; verhalen                       1456
levensbeschrijvingen                          436
autobiografieën                              291
gedichten ; oorspr. - Nederlands              216
reisverhalen                                  134
levenskunst                                   133
prentenboeken                                 118
geloofsleven                                  103
spiritualiteit                                 88
voetbal                                        79
jeugdboeken ; informatie - biologie            79
columns                                        78
Wereldoorlog II ; Nederland                    75
Name: value, dtype: int64


In [6]:
# How many Brinkeys are being assigned for a single entry (concl: varies from 1 to 5 Brinkeys).
df_brinkman_sp = df['brinkman_520x'].str.split('|').str.len()

print('Amount of Brinkeys assigned to a single entry:\n')
print(df_brinkman_sp.value_counts())

Amount of Brinkeys assigned to each entry:

1.0    8789
2.0    2693
3.0     680
4.0      75
5.0       5
Name: brinkman_520x, dtype: int64


## Generate Full-text document corpus

### Generate subject files 
[Annif document corpus formats](https://github.com/NatLibFi/Annif/wiki/Document-corpus-formats)

All files will be saved into /data/vocab directory.

In [41]:
# Create subject vocabulary files.

# Import Brinkman TSV as dict.
dict_subjects = {}
with open('data/brinkmanthesaurus_vocab.tsv', mode='r') as infile:
    reader = csv.reader(infile, delimiter="\t")
    for row in reader:
        dict_subjects[row[1]] = row[0]

# Create new df with only 'ppn' and 'brinkman_520x'
df_asgn_subject = df[['ppn', 'brinkman_520x']]
df_asgn_subject['brinkman_520x'] = df_asgn_subject['brinkman_520x'].str.split('|')
dict_ppn_bk = pd.Series(df.brinkman_520x.values,index=df.ppn).to_dict()

# Maak subject vocab (key) files.

# Create dictionary {ppn : [brinkman_id, brinkman_term]}
dict_assigned_sub = {}
no_direct_sub_match = []
for ppn, asg_subj  in dict_ppn_bk.items():
    if asg_subj not in dict_subjects.keys():
        try:
            asg_subj_split = asg_subj.split(' | ')
            mult_sub = []
            for subj in asg_subj_split:
                if subj in dict_subjects.keys(): 
                    try:
                        mult_sub.append([dict_subjects[subj], subj])
                    except KeyError:
                        pass
                else:
                    no_direct_sub_match.append(ppn)

            dict_assigned_sub[ppn] = mult_sub
        except AttributeError:
            print('[error] ' + ppn +  ' - ' + str(asg_subj))
    else:
        dict_assigned_sub[ppn] = [[dict_subjects[asg_subj], asg_subj]]


# Create .key file for each ppn. Subject vocabulary as TSV.
err = []
if not os.path.exists(os.path.join('data', 'vocab')):
    os.makedirs(os.path.join('data', 'vocab'))
for ppn, subj_id in dict_assigned_sub.items():
    filename = ppn + '.tsv'
    with open(os.path.join(os.path.join('data', 'vocab'), filename), mode='w') as subfile:
        if len(subj_id) > 1:
            for subj_nr in subj_id:
                subfile.write(subj_nr[0] + '\t' + subj_nr[1] + '\n')
        elif len(subj_id) == 1:
            subfile.write(subj_id[0][0] + '\t' + subj_id[0][1])
        else:
            err.append(ppn)

# 'err' is a list with PPN's of problematic entries (no clear brinkman identifier)
# e.g. after split one of the single terms is not found in te vocabulary.

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [39]:
# Remove error.
df = df[df.ppn != '406248214']

In [28]:
# Create summary vocab files.
df_sum = df[['ppn', 'samenvatting_4207']]
dict_sum = pd.Series(df.samenvatting_4207.values,index=df.ppn).to_dict()
for ppn, summ in dict_sum.items():
    filename = ppn + '.txt'
    with open(os.path.join(os.path.join('data', 'vocab'), filename), mode='w') as sumfile:
        sumfile.write(str(summ))

## Split vocab set into Dev, Train and Test set

We will split the full set into:
Dev: 5%
Train: 15%
Test: 80%

3 directories will be created in the vocab dir: 'train', 'dev' and 'test'.

In [44]:
# Split dataset 80-15-5
train, test, dev = np.split(df, [int(.8*len(df)), int(.95*len(df))])

# Create folders
if not os.path.exists(os.path.join(os.path.join('data', 'vocab'), 'train')):
    os.makedirs(os.path.join(os.path.join('data', 'vocab'), 'train'))
if not os.path.exists(os.path.join(os.path.join('data', 'vocab'), 'test')):
    os.makedirs(os.path.join(os.path.join('data', 'vocab'), 'test'))
if not os.path.exists(os.path.join(os.path.join('data', 'vocab'), 'dev')):
    os.makedirs(os.path.join(os.path.join('data', 'vocab'), 'dev'))

# Move selected vocab files to train folder.
for ppn in train['ppn'].tolist():
    os.rename(os.path.join(os.path.join('data', 'vocab'), ppn + '.tsv'), os.path.join(os.path.join(os.path.join('data', 'vocab'), 'train'), ppn + '.tsv'))
    os.rename(os.path.join(os.path.join('data', 'vocab'), ppn + '.txt'), os.path.join(os.path.join(os.path.join('data', 'vocab'), 'train'), ppn + '.txt'))

# Move selected vocab files to test folder.
for ppn in test['ppn'].tolist():
    os.rename(os.path.join(os.path.join('data', 'vocab'), ppn + '.tsv'), os.path.join(os.path.join(os.path.join('data', 'vocab'), 'test'), ppn + '.tsv'))
    os.rename(os.path.join(os.path.join('data', 'vocab'), ppn + '.txt'), os.path.join(os.path.join(os.path.join('data', 'vocab'), 'test'), ppn + '.txt'))
    
# Move selected vocab files to dev folder.
for ppn in dev['ppn'].tolist():
    os.rename(os.path.join(os.path.join('data', 'vocab'), ppn + '.tsv'), os.path.join(os.path.join(os.path.join('data', 'vocab'), 'dev'), ppn + '.tsv'))
    os.rename(os.path.join(os.path.join('data', 'vocab'), ppn + '.txt'), os.path.join(os.path.join(os.path.join('data', 'vocab'), 'dev'), ppn + '.txt'))

## Adding title to summary
We want to test if the outcomes will be better if we add (sub)title information to the summaries. For this we need to create new vocab files.


In [42]:
# How many entries have a subtitle: 5.462 (44,6%)
print('Entries without subtitle: ' + str(len(df[pd.isnull(df['ondertitel_4000'])])))

# How many entries have a title: 12.197 (99,6%)
print('Entries without title: ' + str(len(df[pd.isnull(df['titel_4000'])])))

# Replace NaN by a space (' ') in titel_4000 and ondertitel_4000 (subtitles) before merging data from the three columns.
df['titel_4000'].fillna(' ', axis=0, inplace=True)
df['ondertitel_4000'].fillna(' ', axis=0, inplace=True)

# Remove @ from title.

df['samenvatting_plus_titel'] = df['titel_4000'].str.replace('@', ' ') + ' ' + df['ondertitel_4000'] + ' ' + df['samenvatting_4207']
df

Entries without subtitle: 0
Entries without title: 0


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,maa1,maa2,ppn,jvu_1100,taal_1500_publ,taal_1500_orig,isbn_2000,unesco_1121,unesco_1122,nur_codes_5061,brinkman_520x,brinkman_520x_ppn,onix_7880,prim_auteur_3000,sec_auteur_3011,titel_4000,ondertitel_4000,samenvatting_4207,samenvatting_plus_titel,samenvatting_plus
0,A,Aa,322079640,2015,ned,NaN,9.789054e+12,7,NaN,321,levensbeschrijvingen,075613816,9789054292692,Han/van@Bree$aut$!069567727!Han van Bree 1957-,NaN,De @geest van het Oude Loo,Juliana en haar vriendenkring 1947-1957,In het boek wordt de hofcrisis van 1956 voor h...,De geest van het Oude Loo Juliana en haar vri...,De @geest van het Oude Loo In het boek wordt d...
1,A,Aa,33015673X,2015,ned,fra,9.789490e+12,b,NaN,736,perceptie | esthetiek,075618451 | 075605503,9789490334086,Gilles@Deleuze$aut$!06860873X!Gilles Deleuze 1...,Walter/van der@Star$trl$!125379315!Walter van ...,@Francis Bacon,logica van de gewaarwording,Esthetische analyse van het werk van de Britse...,Francis Bacon logica van de gewaarwording Est...,@Francis Bacon Esthetische analyse van het wer...
2,A,Aa,352655844,2015,ned,NaN,9.789460e+12,7,NaN,698,levensbeschrijvingen,075613816,9789460041228,Anton/van de@Sande$aut$!067525180!Antonius Wil...,NaN,@Prins Frederik der Nederlanden 1797-1881,gentleman naast de troon,"Beschrijving van het leven van prins Frederik,...",Prins Frederik der Nederlanden 1797-1881 gent...,@Prins Frederik der Nederlanden 1797-1881 Besc...
3,A,Aa,352699566,2015,ned,NaN,9.789462e+12,z,NaN,648,vakantieverblijven,075625156,9789462080744,Mieke@Dings$aut$!270022139!Mieke Dings 1979-,NaN,@Tussen tent en villa,het vakantiepark in Nederland 1920-nu,Het vakantiepark: wie heeft er niet weleens ee...,Tussen tent en villa het vakantiepark in Nede...,@Tussen tent en villa Het vakantiepark: wie he...
4,A,Aa,362837317,2015,ned,NaN,9.789047e+12,4,NaN,301,romans en novellen ; oorspr. - Nederlands,075629402,9789046815809,Jan/van der@Mast$aut$!07502943X!Jan van der Ma...,NaN,@Agneta,,"Jacques van Marken (1845-1906), oprichter van ...","Agneta Jacques van Marken (1845-1906), opri...","@Agneta Jacques van Marken (1845-1906), oprich..."
5,A,Aa,363250565,2016,ned,NaN,9.789039e+12,7,NaN,320,autobiografieën | Marokkanen,075657112 | 07566111X,9789038898254,Salaheddine@Benchikhi$aut$!296322547!Salaheddi...,NaN,@Salaheddine punt NL,kom maar op met Nederland,In Salaheddine punt NL vertelt Salaheddine hoe...,Salaheddine punt NL kom maar op met Nederland...,@Salaheddine punt NL In Salaheddine punt NL ve...
6,A,Aa,364403381,2015,ned,NaN,9.789013e+12,f,NaN,822,verbintenissenrecht,075625415,9789013120257,M.M./van@Rossum$aut$!086431471!M.M. van Rossum...,P.H.L.M.@Kuypers$aut$!307225976!Pieter Henri L...,@Garanties in de rechtspraktijk,,Dit boek gaat over de rol van de garantie in d...,Garanties in de rechtspraktijk Dit boek gaa...,@Garanties in de rechtspraktijk Dit boek gaat ...
7,A,Aa,36920042X,2015,ned,NaN,9.789013e+12,f,NaN,826,werkkostenregeling | belastingrecht,390927201 | 075600706,9789013120868,Jacques@Raaijmakers$aut$!138394644!J.H.P.M. Ra...,NaN,@Werkkostenregeling 2.0,(voorheen Belastingvrije vergoedingen),Behandeling van wetgeving zoals die geldt per ...,Werkkostenregeling 2.0 (voorheen Belastingvri...,@Werkkostenregeling 2.0 Behandeling van wetgev...
8,A,Aa,369390687,2015,ned,NaN,9.789025e+12,4,NaN,301,romans en novellen ; oorspr. - Nederlands,075629402,9789025442798,Abdelkader@Benali$aut$!147850665!Abdelkader Be...,NaN,"@Montaigne, een indiaan en de neus van Max Kader",roman,Summary: Max Kader is een gevierd schrijver di...,"Montaigne, een indiaan en de neus van Max Kad...","@Montaigne, een indiaan en de neus van Max Kad..."
9,A,Aa,369539621,2015,ned,NaN,9.789013e+12,f,NaN,822,vermogensrecht,075625733,9789013121629,O.K.@Brahn$aut$$edt$!070525013!O.K. Brahn 1922...,W.H.M.@Reehuis$aut$!072649968!Willem Hendrik M...,@Zwaartepunten van het vermogensrecht,,Summary: In het leven van alledag is het vermo...,Zwaartepunten van het vermogensrecht Summar...,@Zwaartepunten van het vermogensrecht Summary:...


In [43]:
# Vocab files with title data.
df_sum = df[['ppn', 'samenvatting_plus_titel']]
dict_sum = pd.Series(df.samenvatting_plus_titel.values,index=df.ppn).to_dict()
for ppn, summ in dict_sum.items():
    filename = ppn + '.txt'
    with open(os.path.join(os.path.join('data', 'vocab'), filename), mode='w') as sumfile:
        sumfile.write(str(summ))

# Move first set to Annif project folder.
# Run split dataset after this again.